# Spike and Burst Analysis in ABF Files

This notebook walks through the processing of electrophysiological recordings in ABF format. We detect spikes and bursts, extract features from each segment, and visualize the results using UMAP and plotting libraries.

In [ ]:
# Load libraries
import os  # Work with directories and files
import pyabf  # Read ABF files
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis  # Statistical metrics
from scipy.signal import find_peaks  # Detect spikes
%matplotlib widget

# Dimensionality reduction & ML
import umap  # UMAP for nonlinear dimensionality reduction
from sklearn.preprocessing import StandardScaler  # Standardize features

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

# Force Plotly to display in browser
pio.renderers.default = "browser"

## Parameters

- `folder_path`: folder containing ABF files
- `threshold`: voltage threshold for spike detection
- `burst_threshold`: ISI threshold to define bursts
- `fs`: sampling frequency
- `dt`: time step between samples

In [ ]:
folder_path = "bursting"
threshold = -35
burst_threshold = 0.3
fs = 10000
dt = 1 / fs

# Storage for all segment metrics
all_segment_metrics = []

## Processing ABF files
For each ABF file and sweep, we extract spikes and bursts, classify bursts, and compute metrics for each segment.

In [ ]:
# List all ABF files in the folder
abf_files = [f for f in os.listdir(folder_path) if f.endswith(".abf")]

for file_name in abf_files:
    file_path = os.path.join(folder_path, file_name)
    abf = pyabf.ABF(file_path)

    for sweep in range(abf.sweepCount):
        abf.setSweep(sweep)
        time = abf.sweepX  # time vector
        voltage = abf.sweepY  # voltage trace

        # Spike detection
        peaks, _ = find_peaks(voltage, height=threshold)
        spike_times = peaks / fs  # convert indices to time

### Burst detection
- Define bursts as sequences of spikes with interspike intervals below `burst_threshold`.
- Only bursts with ≥2 spikes are considered valid.

In [ ]:
        bursts = []
        if len(spike_times) >= 5:
            isi = np.diff(spike_times)
            current_burst = [spike_times[0]]

            for i in range(1, len(spike_times)):
                if isi[i - 1] < burst_threshold:
                    current_burst.append(spike_times[i])
                else:
                    if len(current_burst) > 1:
                        bursts.append((current_burst[0], current_burst[-1]))
                    current_burst = [spike_times[i]]
            if len(current_burst) > 1:
                bursts.append((current_burst[0], current_burst[-1]))

### Segmentation
- Label each time segment as 'Burst' or 'Non-burst'.
- Add non-burst segments before, between, and after bursts.

In [ ]:
        segments = []
        for start, end in bursts:
            segments.append((start, end, "Burst"))

        if bursts:
            if bursts[0][0] > time[0]:
                segments.insert(0, (time[0], bursts[0][0], "Non-burst"))
            for i in range(len(bursts)-1):
                segments.append((bursts[i][1], bursts[i+1][0], "Non-burst"))
            if bursts[-1][1] < time[-1]:
                segments.append((bursts[-1][1], time[-1], "Non-burst"))
        else:
            segments.append((time[0], time[-1], "Non-burst"))

### Burst type classification
- Compare minimum voltage in burst to surrounding baseline to classify as 'Square Wave', 'Parabolic', or 'Other'.

In [ ]:
        burst_types = []
        for i, (seg_start, seg_end, seg_type) in enumerate(segments):
            if seg_type == "Burst":
                seg_mask = (time >= seg_start) & (time <= seg_end)
                burst_min = np.min(voltage[seg_mask])
                prev_mean = np.nan
                next_mean = np.nan
                if i > 0:
                    prev_end = segments[i-1][1]
                    inter_mask_prev = (time > prev_end) & (time < seg_start)
                    if np.any(inter_mask_prev):
                        prev_mean = np.mean(voltage[inter_mask_prev])
                if i < len(segments)-1:
                    next_start = segments[i+1][0]
                    inter_mask_next = (time > seg_end) & (time < next_start)
                    if np.any(inter_mask_next):
                        next_mean = np.mean(voltage[inter_mask_next])
                inter_mean = np.nanmean([prev_mean, next_mean])
                if np.isnan(inter_mean):
                    burst_type = "Other"
                elif burst_min > inter_mean:
                    burst_type = "Square Wave"
                elif burst_min < inter_mean:
                    burst_type = "Parabolic"
                else:
                    burst_type = "Other"
            else:
                burst_type = "Non-burst"
            burst_types.append(burst_type)

### Metrics extraction
- Compute statistics for voltage and derivative.
- Count peaks and mean ISI.
- Save all metrics in a list.

In [ ]:
        for j, (seg_start, seg_end, seg_type) in enumerate(segments):
            seg_mask = (time >= seg_start) & (time <= seg_end)
            seg_voltage = voltage[seg_mask]
            duration = seg_end - seg_start
            burst_type = burst_types[j]

            seg_peaks, _ = find_peaks(seg_voltage, height=threshold)
            num_peaks = len(seg_peaks)
            mean_isi = np.mean(np.diff(seg_peaks / fs)) if num_peaks >= 2 else 0

            if len(seg_voltage) < 2:
                mean_val = std_val = min_val = max_val = skew_val = kurt_val = area_val = 0
            else:
                mean_val = np.mean(seg_voltage)
                std_val = np.std(seg_voltage)
                min_val = np.min(seg_voltage)
                max_val = np.max(seg_voltage)
                skew_val = skew(seg_voltage)
                kurt_val = kurtosis(seg_voltage)
                area_val = np.trapezoid(seg_voltage, dx=dt)

            if len(seg_voltage) < 2:
                mean_d = std_d = min_d = max_d = skew_d = kurt_d = 0
            else:
                deriv = np.diff(seg_voltage) / dt
                mean_d = np.mean(deriv)
                std_d = np.std(deriv)
                min_d = np.min(deriv)
                max_d = np.max(deriv)
                skew_d = skew(deriv)
                kurt_d = kurtosis(deriv)

            all_segment_metrics.append([
                file_name, sweep, seg_type, burst_type, duration,
                num_peaks, mean_isi,
                mean_val, std_val, min_val, max_val, skew_val, kurt_val, area_val,
                mean_d, std_d, min_d, max_d, skew_d, kurt_d
            ])

## Create DataFrame and save results

In [ ]:
df_segments = pd.DataFrame(all_segment_metrics, columns=[
    "File_Name", "Sweep", "Segment_Type", "Burst_Type", "Duration",
    "Num_Peaks", "Mean_ISI",
    "Mean", "Std", "Min", "Max", "Skewness", "Kurtosis", "Area",
    "Mean_Deriv", "Std_Deriv", "Min_Deriv", "Max_Deriv", "Skewness_Deriv", "Kurtosis_Deriv"
])
df_segments.to_csv("segment_voltage_metrics_with_ISI_and_peaks.csv", index=False)

## UMAP dimensionality reduction and visualization

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
df = pd.read_csv("segment_voltage_metrics_with_ISI_and_peaks.csv")
header = list(df)[4:]
X = np.array(df[header])
y = np.array(df.Burst_Type)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
reducer = umap.UMAP(n_components=2, random_state=42)
embedding = reducer.fit_transform(X_scaled)
df_umap = pd.DataFrame(embedding, columns=["UMAP1", "UMAP2"])
df_umap["Burst_Type"] = y

sns.scatterplot(
    data=df_umap, x="UMAP1", y="UMAP2",
    hue="Burst_Type", palette="tab10", alpha=0.7
)
plt.title("UMAP segments metrics")
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.show()

## Histogram of segment durations

In [ ]:
sns.set(style="whitegrid", context="notebook", rc={'figure.figsize':(10,6)})
sns.histplot(
    data=df_segments,
    x="Duration",
    hue="Burst_Type",
    bins=30,
    kde=True,
    palette="tab10",
    alpha=0.6
)
plt.title("Distribution of the segments duration")
plt.xlabel("Duration (s)")
plt.ylabel("Count")
plt.show()

## Interactive UMAP visualization with Plotly

In [ ]:
fig = px.scatter(
    df_umap, x="UMAP1", y="UMAP2",
    color="Burst_Type",
    title="UMAP"
)
pio.renderers.default = "browser"
fig.show()